In [2]:
import pandas as pd
import cPickle as pickle
from numpy import isnan
from nltk.corpus import stopwords

from textblob import Word
from textblob.base import BaseTokenizer
from textblob import TextBlob
from textblob import Blobber
from textblob.tokenizers import WordTokenizer

In [3]:
#scores=pd.read_csv("Assignment 3 Sentiment Scores (2).csv")
def repLace (x,old, new):
    fixed = x.replace(old,new)
    return fixed
def repLaces (x,old, new):
    fixed = x.lower().replace(old,new)
    return fixed
data=pd.read_csv("Assignment 3 Edmunds Posts.csv", usecols=[0])
data['Posts'] = data['Posts'].map(lambda x: repLace(x,"LexusES","ES"))
data['Posts'] = data['Posts'].map(lambda x: repLace(x,"LexusLS","LS"))
data['Posts'] = data['Posts'].map(lambda x: repLace(x,"LexusRX","RX"))
data['Posts'][4]
# "LexusES dlfja;skdfjalsj".replace("LexusES","LS")

'Which still didn\'t amount to beating their competitors from BMW and Mercedes. Period. Neither IS nor GS were intended to beat 3, 5, C or E on their own.  IS and GS were performance extensions of ES and RX mainstream cars.  ES and RX competed against 325, 525, C240, C280, E320 and their wagon variants; IS and GS competed against 3Series , 530i, 545i, C320, and E500.  If anything, one might make the argument that BMW and MB copy each other\'s product lineups, replete with almost one-on-one sedan and wagon offerings,  whereas Lexus took quite a different route. The CLS competitors are rumored and upcoming, Is that your way of backtracking what you said earlier??  But you are the one spreading the "rumors," whether that rumor has any substance or not. AMG vs M? What does that have to do with anything? AMG is obviously an MB attempt to copy the success BMW\'s M division.  They compete for the similar clientelles; just look at the numerous comparos.  Your argument splitting hair about AMG 

In [264]:
tb=Blobber(tokenizer=WordTokenizer())

In [265]:
#data['tokens']=data['Posts'].apply(lambda x: tb(str(x)).tokenize())

In [3]:
sw=stopwords.words('english')

sw.remove('not')

def doItAllFunction (xstring):
    global sw
    tokens=list(TextBlob(str(xstring)).words)
    removeStopWords=[word for word in tokens if word.lower() not in sw]
    lemmaed=[Word(w).lemmatize() for w in removeStopWords]
    lowercase=[word.lower() for word in lemmaed]
    return lowercase

In [267]:
data['tokens_clean']=data['Posts'].apply(doItAllFunction)

In [268]:
models_skinny=["es","ls","rx","a8","a6","3series","5series","7series","xj","sclass"] #lexusls, lexuses

In [269]:
models=[" es "," ls "," rx "," a8 "," a6 "," 3series "," 5series "," 7series "," xj "," sclass "] #lexusls, lexuses

In [270]:
def finder (tokensList, modelList, numberWords):
    blanklist=[]
    for i in xrange(len(tokensList)):
        if tokensList[i] in modelList:
            blanklist.append(tokensList[i-numberWords:i+1+numberWords])
    return blanklist

In [271]:
data['model_tokens']=data['tokens_clean'].apply(lambda x: finder(x,models_skinny, 2))

In [4]:
class DoItAll(BaseTokenizer):
    sw=sw
    def __self__(self):
        pass
    def tokenize(self, text):
        tokens=list(TextBlob(str(text),tokenizer=WordTokenizer()).words)
        removeStopWords=[word for word in tokens if word.isalpha() and word.lower() not in sw]
        lemmaed=[Word(w).lemmatize() for w in removeStopWords]
        lowercase=[word.lower() for word in lemmaed]
        return lowercase

In [272]:
data['model_strings']=data['model_tokens'].apply(lambda x: [' '.join(inner) for inner in x])

In [273]:
df=pd.DataFrame(columns=models).join(data['model_strings'], how="outer")

In [274]:
for index,l in enumerate(df['model_strings']):
    for string in l: #looping over all the strings in model_strings
            for model in models: #loop over all the models in model list
                if model in string: #check if model is in a particular list
                    if isnan(df[model].iloc[index]): #correcting for neutral
                        df[model].iloc[index]=0
                    df[model].iloc[index]+=tb(string).sentiment[0]      

In [275]:
df['model_strings'][4]

[u'performance extension es rx mainstream',
 u'extension es rx mainstream car',
 u'mainstream car es rx competed',
 u'car es rx competed 325',
 u'gs competed 3series 530i 545i',
 u'headed much sclass sale worldwide',
 u'build lexus ls one lexus',
 u'build mercedes sclass according not',
 u'not necessarily 7series point anyone',
 u'anyone make sclass except mb']

In [276]:
df.to_csv("outputv3.csv")

In [277]:
df['model_strings'][4]

[u'performance extension es rx mainstream',
 u'extension es rx mainstream car',
 u'mainstream car es rx competed',
 u'car es rx competed 325',
 u'gs competed 3series 530i 545i',
 u'headed much sclass sale worldwide',
 u'build lexus ls one lexus',
 u'build mercedes sclass according not',
 u'not necessarily 7series point anyone',
 u'anyone make sclass except mb']